# LOADING IMAGES

### Paths for the input files

In [ ]:
path_data_train = "BSR_bsds500\BSR\BSDS500\data\images\\train"
path_data_test = "BSR_bsds500\BSR\BSDS500\data\images\\test"
path_data_validate = "BSR_bsds500\BSR\BSDS500\data\images\\val"

# path_data_train = "BSR_bsds500/BSR_bsds500/BSR/BSDS500/data/images/train"
# path_data_test = "BSR_bsds500/BSR_bsds500/BSR/BSDS500/data/images/test"
# path_data_validate ="BSR_bsds500/BSR_bsds500/BSR/BSDS500/data/images/val"

### Converting images to greyscale

In [ ]:
import numpy as np
import cv2
import os
def preprocess_input(input_file_path):
    input_file_list = os.listdir(input_file_path)
    img_data_list = []
    for file in input_file_list:
        img = cv2.imread(input_file_path+'\\'+file)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_data_list.append(gray)
    print ("Converted "+str(len(img_data_list))+ " images to greyscale.")
    inp_data = np.array(img_data_list)
    print(inp_data.shape)
    return inp_data

### Storing images in lists

In [ ]:
data_train_list = preprocess_input(path_data_train)
data_test_list = preprocess_input(path_data_test)
data_validate_list = preprocess_input(path_data_validate)

## Creating patches adding noise

### Sliding window technique

In [ ]:
def sliding_window(image, stepSize, windowSize):
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

### Extract clean and noisy patches

In [ ]:
import itertools
def get_patches(input_list, size):
    print("Extracting patches...")
    print(len(input_list))
    clean_patch_list = []
    noisy_patch_list = []
    for clean_image in itertools.islice(input_list, 0, len(input_list)):
        for(x,y, window) in sliding_window(clean_image, stepSize=size, windowSize=(size,size)):
            if window.shape[0] != size or window.shape[1] != size:
                continue
            clean_patch=window
            if clean_patch.shape[0] != size or clean_patch.shape[1] != size:
                continue
            clean_patch_list.append(clean_patch)

            noisy_patch = clean_patch.copy()
            noisy_patch = np.add(noisy_patch, ( 25 * np.random.random(noisy_patch.shape)))
            noisy_patch = np.clip(noisy_patch,0,255.0)
            noisy_patch_list.append(noisy_patch)
                
    clean_patches = np.asarray([arr.flatten().astype(np.float32) for arr in clean_patch_list])
    noisy_patches = np.asarray([arr.flatten().astype(np.float32) for arr in noisy_patch_list])
    print("Total patches extracted: " + str(clean_patches.shape))
    print("Single patche size: " + str(clean_patches[1].shape))

    return (clean_patches, noisy_patches)  

In [ ]:
train_patches = np.asarray(get_patches(data_train_list, 8))
test_patches = np.asarray(get_patches(data_test_list, 8))
val_patches = np.asarray(get_patches(data_validate_list, 8))

train_patches_16 = np.asarray(get_patches(data_train_list, 16))
test_patches_16 = np.asarray(get_patches(data_test_list, 16))
val_patches_16 = np.asarray(get_patches(data_validate_list, 16))

### Noramlize the patches

In [ ]:
# INPUT DATA 8x8 FOR THE MODEL IN PROPER SHAPE
x_train_clean = train_patches[0] 
x_train_clean = x_train_clean/255.0
x_train_clean = x_train_clean.reshape(((len(x_train_clean)),64))

x_train_noisy = train_patches[1] 
x_train_noisy = x_train_noisy/255.0
x_train_noisy = x_train_noisy.reshape(((len(x_train_noisy)),64))

x_test_clean = test_patches[0] 
x_test_clean = x_test_clean/255.0
x_test_clean = x_test_clean.reshape(((len(x_test_clean)),64))

x_test_noisy = test_patches[1] 
x_test_noisy = x_test_noisy/255.0
x_test_noisy = x_test_noisy.reshape(((len(x_test_noisy)),64))

###################

# INPUT DATA 16x16 FOR THE MODEL IN PROPER SHAPE
x_train_clean_16 = train_patches_16[0] 
x_train_clean_16 = x_train_clean_16/255.0
x_train_clean_16 = x_train_clean_16.reshape(((len(x_train_clean_16)),256))

# MODEL

The autoencoder consists of an input layer and a single encoding layer and a single decoding layer. The model is trained with clean images as input and clean images as output. The weights of the encoding layer are stored after training.
    This function returns the encoder model that maps the input layer to its encoded representation, and the weights of the encoder layer.

In [ ]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K
K.set_image_dim_ordering('tf')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,
                                         write_graph=True, write_images=True)

In [ ]:
def my_autoencoder(i_dim, e_dim, train, name):
    #input dimension
    input_dim = i_dim*i_dim
    #encoding dimension
    encoding_dim = e_dim*e_dim
    
    #AUTOENCODER_MODEL

    #input layer
    input_layer = Input(shape=(input_dim,))
    
    #encoded layer 
    e_name = str(name)+'_en_d'
    encoding_layer = Dense(encoding_dim, activation='relu', kernel_initializer='random_normal', name=e_name)(input_layer)
    
    #decoding layer
    d_name = str(name)+'_de_d'
    decoding_layer = Dense(input_dim, activation='sigmoid', kernel_initializer='random_normal', name=d_name)(encoding_layer)
    
    #model
    auto_1 = Model(input_layer, decoding_layer)
    
    #compile
    auto_1.compile(optimizer='adadelta', loss ='mean_squared_error', metrics=['accuracy'])
    
    #train
    auto_1.fit(train, train,
          epochs = 40,
          verbose =1,
          batch_size = 100,
          shuffle = True,
          validation_split=0.33,
          callbacks=[tbCallBack])

    #save all weights
    w_name = 'encoded'+str(name)+'weights.h5'
    auto_1.save_weights(w_name) # save weights
    
    #save weights of encoder layer
    weights = auto_1.layers[1].get_weights()
       
    #ENCODER MODEL : mapping input to the encoded representation
    encoder = Model(input_layer, encoding_layer)
    #load weights of encoding layer from the trained model (weights of layers with same name loaded)
    encoder.load_weights(w_name, by_name=True) 
    #compile
    encoder.compile(optimizer='adadelta', loss = 'mean_squared_error')

    #return the ecnoder model for feed forwarding and the weights of the encoder layer to use in SSDA
    return (encoder, weights) 

### AUTOENCODER MODEL 1

In [ ]:
# FIT MODEL 1
autoencoder1 = my_autoencoder(8, 16, x_train_clean, 1)
# save weights of encoder layer from model 1
wt1 = autoencoder1[1]

![](loss_1.JPG)

<center>loss vs epoch (autoencoder 1)</center>

### FEED FORWARDING

The encoder model obtained from the previous step is fed with the clean and noisy input.

In [ ]:
# FEED FORWARDING 1
encoded_imgs_clean = autoencoder1[0].predict(x_train_clean)
encoded_imgs_noisy = autoencoder1[0].predict(x_train_noisy)

### AUTOENCODER MODEL 2

In [ ]:
# FIT MODEL
autoencoder2 = my_autoencoder(16, 32, x_train_clean_16, 2)
# save weights of encoder layer from model 2
wt2 = autoencoder2[1]

![](loss_2.JPG)

<center>loss vs epoch (autoencoder 2)</center>

### FEED FORWARDING

The encoder model obtained from the second autoencoder is fed with the clean and noisy output of the first autoencoder's feed forward step.

In [ ]:
# PREDICTIONS / TESTS
encoded_imgs_clean2 = autoencoder2[0].predict(encoded_imgs_clean)
encoded_imgs_noisy2 = autoencoder2[0].predict(encoded_imgs_noisy)

## SSDA

The SSDA has a input layer, 2 encoding layers that take the input from 8x8 to 16x16 and 16x16 to 32x32. The weights of these layers are initialised using the weights of the first two autoencoders stored earlier. Then comes two decoding layers that bring the 32X32 image to 16x16 and then to 8x8.

In [ ]:
def ssda(w1, w2, noisy, clean):
  
    #AUTOENCODER_MODEL

    #input layer
    input_layer = Input(shape=(8*8,))
 
    #encoded layer 
    encoding_layer_8_16 = Dense(16*16, activation='relu', name='ssd_e_16')(input_layer)
    encoding_layer_16_32 = Dense(32*32, activation='relu', name='ssd_e_32')(encoding_layer_8_16)

    #decoding layer
    decoding_layer_32_16 = Dense(16*16, activation='sigmoid', name='ssd_d_32')(encoding_layer_16_32)
    decoding_layer_16_8 = Dense(8*8, activation='sigmoid', name='ssd_d_16')(decoding_layer_32_16)
    
    #MODEL
    ssda_model = Model(inputs=[input_layer], outputs=[decoding_layer_16_8])
    
    ssda_model.layers[1].set_weights(w1)
    ssda_model.layers[2].set_weights(w2)
    
    ssda_model.compile(optimizer='adadelta', loss = 'mean_squared_error')
    
    ssda_model.fit(noisy, clean,
          epochs = 30,
          verbose =2,
          batch_size = 1000,
          shuffle = True,
          validation_split=0.33,
          callbacks=[tbCallBack])  
    
    ssda_model.save_weights('ssda_wts')
    
    return ssda_model
    

### FIT THE MODEL

In [ ]:
model = ssda(wt1, wt2, x_train_noisy, x_train_clean)

![](loss_ssda.JPG)

<center>loss vs epoch (SSDA)</center>

# TESTING

In [ ]:
# TEST USING NOISY INPUT
img = model.predict(x_test_noisy)

# saving the output to a file for future use
import pickle
with open('outfile', 'wb') as fp:
    pickle.dump(img, fp)

### Plotting a few random patches

In [ ]:
import matplotlib.pyplot as plt
import skimage.measure as sk

n = 10 
k = 50
plt.figure(figsize=(20, 4))

for i in range(n):
    # display original
    ax = plt.subplot(3, n, i + 1)
    ax.set_title('Patch : ' + str(i*k))
    plt.imshow(x_test_clean[i*k].reshape(8,8))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

for i in range(n):
    # display noisy
    ax = plt.subplot(3, n, i + 1 +n)
    ax.set_title('PSNR: ' + str(round(sk.compare_psnr(x_test_clean[i*k],x_test_noisy[i*k]),2)))
    plt.imshow(x_test_noisy[i*k].reshape(8,8))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
for i in range(n):
    # display deniosed
    ax = plt.subplot(3, n, i + 1 + 2*n)
    ax.set_title('PSNR: ' + str(round(sk.compare_psnr(x_test_clean[i*k],itemlist[i*k]),2)))
    plt.imshow(itemlist[i*k].reshape(8,8))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.tight_layout()
plt.show()

![](Capture3.JPG)

<center>Top: Clean image, Middle: Noisy Input, Bottom: Denoised Image</center>

In [ ]:
# Clean test image
plt.imshow(data_validate_list[10])
plt.show()

In [ ]:
# Noisy test image
plt.imshow(np.add(data_validate_list[10], 25*np.random.random(data_validate_list[10].shape)))
plt.show()

In [ ]:
# Convering test images to patches
test_image = np.asarray(get_patches(data_test_list[10:11], 8))
test_image_clean = test_image[0]
test_image_clean = test_image_clean/255.0
test_image_clean = test_image_clean.reshape(((len(test_image_clean)),64))

test_image_noisy = test_image[1]
test_image_noisy = test_image_noisy/255.0
test_image_noisy = test_image_noisy.reshape(((len(test_image_noisy)),64))

In [ ]:
test_res = model.predict(test_image_noisy)

In [ ]:
with open('test_op', 'wb') as fp:
    pickle.dump(test_res, fp)

In [ ]:
# # from file
plt.subplot(131)
plt.imshow(data_test_list[10][0:8,8:16].reshape(8,8))
plt.colorbar(fraction=0.046, pad=0.04)

# # from patch noisy
plt.subplot(132)
plt.imshow(test_image_noisy[1].reshape(8,8)*255)
plt.colorbar(fraction=0.046, pad=0.04)

# # from patch denoised
plt.subplot(133)
plt.imshow(test_res[1].reshape(8,8)*255)
plt.colorbar(fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

![](test.JPG)

<center>Patch no.2 from the test image</center>